In [2]:
import pandas as pd
df=pd.read_csv(r'C:\Users\admin\Documents\PREPARATION\GiveMeSomeCredit\cs-test.csv')
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [2]:
df.isnull().sum()

Unnamed: 0                                   0
SeriousDlqin2yrs                        101503
RevolvingUtilizationOfUnsecuredLines         0
age                                          0
NumberOfTime30-59DaysPastDueNotWorse         0
DebtRatio                                    0
MonthlyIncome                            20103
NumberOfOpenCreditLinesAndLoans              0
NumberOfTimes90DaysLate                      0
NumberRealEstateLoansOrLines                 0
NumberOfTime60-89DaysPastDueNotWorse         0
NumberOfDependents                        2626
dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101503 entries, 0 to 101502
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            101503 non-null  int64  
 1   SeriousDlqin2yrs                      0 non-null       float64
 2   RevolvingUtilizationOfUnsecuredLines  101503 non-null  float64
 3   age                                   101503 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  101503 non-null  int64  
 5   DebtRatio                             101503 non-null  float64
 6   MonthlyIncome                         81400 non-null   float64
 7   NumberOfOpenCreditLinesAndLoans       101503 non-null  int64  
 8   NumberOfTimes90DaysLate               101503 non-null  int64  
 9   NumberRealEstateLoansOrLines          101503 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  101503 non-null  int64  
 11  

In [6]:
age_bins = pd.cut(df['age'], bins=5)
age_bins

0           (37.6, 54.2]
1           (54.2, 70.8]
2           (54.2, 70.8]
3           (37.6, 54.2]
4         (20.917, 37.6]
               ...      
101498    (20.917, 37.6]
101499    (20.917, 37.6]
101500      (54.2, 70.8]
101501      (54.2, 70.8]
101502    (20.917, 37.6]
Name: age, Length: 101503, dtype: category
Categories (5, interval[float64, right]): [(20.917, 37.6] < (37.6, 54.2] < (54.2, 70.8] < (70.8, 87.4] < (87.4, 104.0]]

In [7]:
df['NumberOfDependents_missing'] = df['NumberOfDependents'].isnull().astype(int)
df['NumberOfDependents'] = df.groupby(age_bins)['NumberOfDependents'].transform(lambda x:x.fillna(x.median()))

C:\Users\admin\AppData\Local\Temp\ipykernel_22380\3238020934.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['NumberOfDependents'] = df.groupby(age_bins)['NumberOfDependents'].transform(lambda x:x.fillna(x.median()))


In [8]:
df['MonthlyIncome_missing'] = df['MonthlyIncome'].isnull().astype(int)
df['MonthlyIncome'] = df.groupby(age_bins)['MonthlyIncome'].transform(lambda x:x.fillna(x.median()))

C:\Users\admin\AppData\Local\Temp\ipykernel_22380\3521791555.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['MonthlyIncome'] = df.groupby(age_bins)['MonthlyIncome'].transform(lambda x:x.fillna(x.median()))


In [9]:
df_test = df.drop(columns=['SeriousDlqin2yrs'])

In [11]:
# Tổng số lần trễ hạn
df_test['TotalDelinquency'] = (
    df_test['NumberOfTime30-59DaysPastDueNotWorse'] +
    df_test['NumberOfTime60-89DaysPastDueNotWorse'] +
    df_test['NumberOfTimes90DaysLate']
)

In [13]:
import joblib
df_test['Id'] = range(1, len(df_test) + 1)
X_test = df_test.drop(columns=['Id'])

# Load mô hình
model = joblib.load('xgb_model.pkl')

# Predict xác suất
y_test_proba = model.predict_proba(X_test)[:, 1]

# Tạo file submit
submission = pd.DataFrame({
    'Id': df_test['Id'],
    'Probability': y_test_proba
})
submission.to_csv("submission.csv", index=False)
print(" Đã tạo file submission.csv để nộp lên Kaggle")

 Đã tạo file submission.csv để nộp lên Kaggle
